### Implementación de una red neuronal con dos capas ocultas y mini-batch (indexada)

In [138]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib

Using matplotlib backend: Qt5Agg


In [159]:
def sigmoid(z):
    out = 1/(1 + np.exp(-z))
    return out

def trainNN_mini_batch(X, y, lr=0.01, amt_epochs=100, n_batchs=16):
    
    global J_train
    global J_valid
    
    # Inicializamos aleatoriamente los parámatros
    #============================================
    W1 = np.random.rand(2,3)
    W2 = np.random.rand(3,2)
    W3 = np.random.rand(2,1)
    b1 = np.random.rand(3,1)
    b2 = np.random.rand(2,1)
    b3 = np.random.rand()
    
    # Separo una parte de los datos para validation
    #----------------------------------------------
    permuted_idxs = np.random.permutation(X.shape[0])
    train_ids = permuted_idxs[0:int(0.8 * X.shape[0])]
    valid_ids = permuted_idxs[int(0.8 * X.shape[0]): X.shape[0]]
    X_train = X[train_ids]
    X_valid = X[valid_ids]
    y_train = y[train_ids]
    y_valid = y[valid_ids]
    
    # Algunas valores que nos van a servir
    #-------------------------------------
    n = X_train.shape[0]m
    nv = X_valid.shape[0]
    batch_size = int(len(X_train) / n_batchs) # Paso o tamaño en muestras del batch
    batch_idxs = np.arange(0, n-n_batchs, n_batchs)    # Indices para tomar las muestras de los batchs
    
    # Inicializo los vectores de datos
    J_train = []
    J_valid = []
    
    # Primer for con las iteraciones
    #-------------------------------
    for i in range (amt_epochs):
        
        # Inicializo la integral de pesos del batch
        J_int = 0
        
        # Permutamos los índices de los datos de entrenamiento (mezclamos)
        idx = np.random.permutation(X_train.shape[0])
        X_train = X_train[idx]
        y_train = y_train[idx]
        
        # Segundo for para el batch armado
        for j in range(n_batchs):
            
            # Armo el mini batch de la iteración en cuestión
            X_batch = X_train[batch_idxs+j,:]
            y_batch = y_train[batch_idxs+j]
            
            # Forward
            #--------
            z1 = W1.T @ X_batch.T + b1
            a1 = sigmoid(z1)
            z2 = W2.T @ a1 + b2
            a2 = sigmoid(z2)
            z3 = W3.T @ a2 + b3          
            y_hat = sigmoid(z3)
            # Calculo el error del pasaje "forward" en esta corrida
            J = (1/batch_size) * np.sum(np.power(y_batch-y_hat,2))
            
            # Backpropagation
            #----------------
            # Output layer
            dz3 = -2*(y_batch-y_hat) * y_hat * (1-y_hat)
            grad_W3 = (1/batch_size) * dz3 @ a2.T
            grad_b3 = (1/batch_size) * np.sum(dz3,axis=1,keepdims=True)
            
            # Hidden layer: Layer 2
            dz2 = np.multiply(W3 @ dz3, (a2*(1-a2)))
            grad_W2 = (1/batch_size) * dz2 @ a1.T
            grad_b2 = (1/batch_size) * np.sum(dz2,axis=1,keepdims=True)
            
            # Hidden layer: Layer 1
            dz1 = np.multiply(W2 @ dz2, (a1*(1-a1)))
            grad_W1 = (1/batch_size) * dz1 @ X_batch
            grad_b1 = (1/batch_size) * np.sum(dz1,axis=1,keepdims=True)
            
            # Updates
            W3 = W3 - lr * grad_W3.T * J
            W2 = W2 - lr * grad_W2.T * J
            W1 = W1 - lr * grad_W1.T * J
            b3 = b3 - lr * grad_b3 * J
            b2 = b2 - lr * grad_b2 * J
            b1 = b1 - lr * grad_b1 * J
            
            J_int = J_int + J

        # Calculo el error sobre el set de validación
        z1 = W1.T @ X_valid.T + b1
        a1 = sigmoid(z1)
        z2 = W2.T @ a1 + b2
        a2 = sigmoid(z2)
        z3 = W3.T @ a2 + b3          
        y_hat = sigmoid(z3)
        
        J_valid.append( (1/nv) * np.sum(np.power(y_valid-y_hat,2)) )
        
        # Guardo el valor de la función de costo a la salida de cada batch
        J_train.append(J_int/n_batchs)
        
    return W1,W2,W3,b1,b2,b3

def testNetwork(W1,W2,W3,b1,b2,b3,X_test):
    n = X_test.shape[0]
    y_hat = np.zeros((n,1), dtype=float)
    for i in range(n):
        z1 = W1.T @ X_test.T + b1
        a1 = sigmoid(z1)
        z2 = W2.T @ a1 + b2
        a2 = sigmoid(z2)
        z3 = W3.T @ a2 + b3          
        y_hat = sigmoid(z3)
    
    return y_hat

In [178]:
# Cargamos los datos de entrenamiento
#------------------------------------
data = pd.read_csv("train_data.csv",skiprows = 0)
data = np.array(data)

# Entrenamos la red
#------------------
X = data[:,:2]
y = data[:,2]
W1,W2,W3,b1,b2,b3 = trainNN_mini_batch(X, y, lr=0.25, amt_epochs=30000, n_batchs = 24)

# Ploteamos la evolución del costo en el aprendizaje
#---------------------------------------------------
plt.plot(J_train,'r')
plt.plot(J_valid,'b')
plt.gca().set_title('Evolución función de costo')
plt.legend(['Error de entrenamiento','Error de validación'])
plt.xlabel('Iteraciones')
plt.ylabel('Costo')
plt.show()

In [179]:
# Cargamos los datos de testeo
#-----------------------------
data = pd.read_csv("test_data.csv",skiprows = 0)
data = np.array(data)
X = data[:,:2]
y = data[:,2]

# Testeamos la red con los pesos aprendidos
#------------------------------------------
y_guess = testNetwork(W1,W2,W3,b1,b2,b3,X)
print(y_guess)
print(y)
print(np.sum(y-y_guess))

[[0.98823069 0.04565422 0.05730226 0.02347417 0.01422017 0.99100745
  0.02332772 0.00582104 0.09245874 0.9835805  0.96336013 0.98688452
  0.00698958 0.40828515 0.07194554 0.01562974 0.97642829 0.98166711
  0.3669129  0.99348651 0.99685568 0.02308176 0.46783242 0.00564427
  0.98714483 0.9614045  0.98290925 0.93253562 0.00880174 0.95578229
  0.09386216 0.97044178 0.94693808 0.98913756 0.0376354  0.97052663
  0.98482321 0.9981072  0.97208409 0.15207439 0.00578608 0.0389664
  0.0242606  0.02797548 0.97886972 0.00885861 0.85538623 0.76946526
  0.01124647 0.95903501 0.00622604 0.24559102 0.00658242 0.0124565
  0.93393223 0.60534114 0.00856335 0.95813566 0.61363593 0.00650734
  0.6864087  0.98252032 0.0072688  0.97284577 0.02645906 0.97075375
  0.96092772 0.00747401 0.01375153 0.00996646 0.97457348 0.7724674
  0.99042837 0.01908088 0.98922308 0.98070241 0.02284078 0.9680797
  0.32864542 0.75114637 0.876464   0.98489413 0.8977348  0.94686783
  0.98083494 0.43362743 0.98750629 0.04197907 0.9861

#### Conclusiones
- Se comprueba que la función de costo va disminuyendo con las iteraciones.
- Se observan dos escalones (de la función de costo), uno inicial y otro alrededor de la iteración 500 (en algunas corridas estos dos pasos son más marcados)
- Al testear la red no se equivoca en ninguna salida...pero tengo dudas sobre:
    - ¿Por qué no llega la sigmoidea a saturar? (no llego a tener 1s y 0s en y_guess)
    - ¿Por qué se precisa de un learning rate tan alto?...bah...¿es alto?
    - ¿Por qué se precisa de tantas iteraciones?
    - ¿Cómo elegiría la cantidad de batchs? en 8 más o menos funciona
    - ¿Me estoy equivocando en algo?